In [2]:
import os, time
from pathlib import Path
from operator import add
import numpy as np
from glob import glob
import glob
import cv2
from tqdm import tqdm
import torch
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score

import sys
sys.path.append('/zhome/ac/d/174101/thesis/src')
from utils.ResNet import ResNet

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

cpu
torch.Size([1, 256, 56, 56])
torch.Size([1, 1000])


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
def calculate_metrics(y_true, y_pred,threshold):
    # Ground truth
    y_true = y_true.cpu().numpy()
    y_true = y_true > threshold
    y_true = y_true.astype(np.uint8)
    y_true = y_true.reshape(-1)

    # Prediction
    y_pred = y_pred.cpu().numpy()
    y_pred = y_pred > threshold
    y_pred = y_pred.astype(np.uint8)
    y_pred = y_pred.reshape(-1)

    score_jaccard = jaccard_score(y_true, y_pred)
    score_f1 = f1_score(y_true, y_pred)
    score_recall = recall_score(y_true, y_pred)
    score_precision = precision_score(y_true, y_pred)
    score_acc = accuracy_score(y_true, y_pred)

    return [score_jaccard, score_f1, score_recall, score_precision, score_acc]

if __name__ == "__main__":
    
    # Directories
    base_path = '/work3/s220243/Thesis'
    create_dir(base_path / "results")

    #Clean the 'results' dir - optional
    dir = (base_path / "results")
    filelist = glob.glob(os.path.join(dir, "*"))
    #for f in filelist:
        #os.remove(f)

    # Load dataset
    test_x = list((base_path / "data_split_lbp/val/images/").glob("*.jpeg"))
    test_y = list((base_path / "data_split_lbp/val/masks/").glob("*.jpeg"))
    
    test_x.sort(key=sort_path_list)
    test_y.sort(key=sort_path_list)

    # Hyperparameters
    H = 224
    W = 224
    size = (W, H)
    checkpoint_path = base_path / "models/fungi_model_base_B32_E1000_lr1.000e-04_SGD_CrossEntropy"

    # Load checkpoint
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model_parameters={}
    model_parameters['resnet50'] = ([64,128,256,512],[3,4,6,3],4,True)
    model = ResNet(model_parameters['resnet50'] , in_channels=3, num_classes=num_classes)
    model = model.to(device)
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    model.eval()

    metrics_score = [0.0, 0.0, 0.0, 0.0, 0.0]
    time_taken = []
    pred_y_plot=[]
    img_plot=[]
    mask_plot=[]

    for i, (x_file, y_file) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
        # Extract the name
        x_filename = x_file.stem
        y_filename = y_file.stem

        """ Reading image """
        image = cv2.imread(str(x_file), cv2.IMREAD_COLOR) ## (512, 512, 3)
        x = np.transpose(image, (2, 0, 1))      ## (3, 512, 512)
        x = x/255.0
        x = np.expand_dims(x, axis=0)           ## (1, 3, 512, 512)
        x = x.astype(np.float32)
        x = torch.from_numpy(x)
        x = x.to(device)


        """ Reading mask """
        mask = cv2.imread(str(y_file), cv2.IMREAD_GRAYSCALE)  ## (512, 512)
        y = np.expand_dims(mask, axis=0)            ## (1, 512, 512)
        y = y/255.0
        y = np.expand_dims(y, axis=0)               ## (1, 1, 512, 512)
        y = y.astype(np.float32)
        y = torch.from_numpy(y)
        y = y.to(device)

        with torch.no_grad():
            """ Prediction and Calculating FPS """
            start_time = time.time()
            pred_y = model(x)
            #pred_y = torch.nn.Softmax(pred_y)
            pred_y = torch.sigmoid(pred_y)
            total_time = time.time() - start_time
            time_taken.append(total_time)


            score = calculate_metrics(y, pred_y, threshold)
            metrics_score = list(map(add, metrics_score, score))
            pred_y = pred_y[0].cpu().numpy()        ## (1, 512, 512)
            pred_y = np.squeeze(pred_y, axis=0)
            #for ploting

            pred_y_plot.append(pred_y)
            img_plot.append(image)
            
            
            pred_y = pred_y > threshold
            pred_y = np.array(pred_y, dtype=np.uint8)

    jaccard = metrics_score[0]/len(test_x)
    f1 = metrics_score[1]/len(test_x)
    recall = metrics_score[2]/len(test_x)
    precision = metrics_score[3]/len(test_x)
    acc = metrics_score[4]/len(test_x)
    print(f"Jaccard: {jaccard:1.4f} - F1: {f1:1.4f} - Recall: {recall:1.4f} - Precision: {precision:1.4f} - Acc: {acc:1.4f}")

    fps = 1/np.mean(time_taken)
    print("FPS: ", fps)

In [ ]:
# Confusion Matrix
confusion_matrix = metrics.confusion_matrix(ground_truth, predictions)

# visualize confusion matrix with sklearn metrics
# define matrix and create labels
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix)#, display_labels = ["Not Spam", "Spam"])

# display matrix
cm_display.plot()
plt.show()

In [ ]:
# Create a confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
sns.heatmap(cm, annot=True,fmt='d', cmap='YlGnBu', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.ylabel('Prediction',fontsize=12)
plt.xlabel('Actual',fontsize=12)
plt.title('Confusion Matrix',fontsize=16)
plt.show()

In [15]:
import sys
import torch
from pathlib import Path
from torchvision.transforms import v2
import torchvision.datasets as datasets
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
sys.path.append('/zhome/ac/d/174101/thesis/src')
from utils.ResNet import ResNet


# Directories
base_path = '/work3/s220243/Thesis'
val_dir = Path(base_path) / 'data_split_resized/validation'

checkpoint_path = Path(base_path) / "models/fungi_model_base_B32_E1000_lr1.000e-04_SGD_CrossEntropy.pth"

# Define transformations
transform = v2.Compose([
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize([0.7129, 0.6080, 0.5386], [0.1165, 0.1868, 0.1702]) #Resized only
        #v2.Normalize([0.5272, 0.5272, 0.5272],[0.2306, 0.2306, 0.2306]) #LBP
])

# Load test data using ImageFolder
image_dataset = datasets.ImageFolder(val_dir, transform=transform)

# Create a DataLoader for the test dataset
val_loader = torch.utils.data.DataLoader(image_dataset, batch_size=1, shuffle=False)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Load model
class_names = image_dataset.classes
num_classes = len(class_names)
#print(num_classes)
model_parameters={}
model_parameters['resnet50'] = ([64,128,256,512],[3,4,6,3],4,True)
model = ResNet(model_parameters['resnet50'] , in_channels=3, num_classes=72)
model = model.to(device)
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.eval()

# Initialize lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through the test_loader
for images, labels in val_loader:
    
    # Perform inference using your model on these images
    with torch.no_grad():
        output = model(images)
        predictions = torch.argmax(F.softmax(output, dim=1), dim=1).cpu().numpy()
    
    # Convert tensors to numpy arrays
    labels = labels.cpu().numpy()
    
    # Append true labels and predicted labels to the lists
    true_labels.extend(labels)
    predicted_labels.extend(predictions)

# Convert lists to numpy arrays for easier calculation of metrics
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

cpu
69


KeyboardInterrupt: 